# Calculate robusteness from pi control runs

In [41]:
# Some definitions

# directory of data
#fdir = '~/Google Drive/professional/research/FARALLON_INSTITUTE_PROJECTS/2020 NOAA MAPP/Climate_extremes_sharedfigsandcode/data/FOR SCATTER PLOT/'
fdir = '/Volumes/GoogleDrive/My Drive/Climate_extremes_sharedfigsandcode/data/Annual_TimeSeries/'

In [42]:
# Modules
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore') #filter some warning messages
import seaborn as sns

In [43]:
def num_coevents(y1, y2, pct1, pct2, runs='picontrol',loc='cclme'):
    # 
    fin1 = fdir+'original ts/'+runs+'_'+loc+'_sst.nc'
    fin2 = fdir+'original ts/'+runs+'_sm.nc'
    modsst = xr.open_dataset(fin1)
    modsst.close()
    modsm2 = xr.open_dataset(fin2)
    modsm2.close()
    
    models = modsm2.model

    # selec periood
    modsst = modsst.sel(year=slice(y1,y2))
    #modsm2 = modsm2.sel(year=slice(y1-1,y2-1))
    modsm2 = modsm2.sel(year=slice(y1,y2))

    
    nev = list()
    mhw = list()
    drt = list()
    for ix,i in enumerate(models):
        #print(ix,i)
        tmp1= modsst.sel(model=i).sst.values
        tmp2= modsm2.sel(model=i).sm.values
    
        # calculate threshold
        mhw_thr = np.nanpercentile(tmp1,pct1)
        drg_thr = np.nanpercentile(tmp2,pct2)
    
        a1 = tmp1>=mhw_thr
        a2 = tmp2<=drg_thr
        
        tmp = np.full((len(modsst.year.values),1),1)
        tmp = tmp[a1*a2]

        tmp1 = np.full((len(modsst.year.values),1),1)
        tmp1 = tmp1[a1]
        tmp2 = np.full((len(modsst.year.values),1),1)
        tmp2 = tmp2[a2]
        
        nev.append(len(tmp)/30*100)
        mhw.append(len(tmp1)/30*100)
        drt.append(len(tmp2)/30*100)
        
    return nev, mhw, drt # sum(nev)

In [44]:
ny = 30
prc1 = 90
prc2 = 10
loc='cclme'
pinev=list()
pim=list()
pid=list()
for i in range(int(500/ny)):
    nev,m0,d0 = num_coevents(i*ny+1, (i+1)*ny, prc1 , prc2, loc=loc)
    pinev.append(nev)
    pim.append(m0)
    pid.append(d0)
pinev,np.mean(pinev),np.mean(pim), np.mean(pid) 

([[3.3333333333333335,
   0.0,
   0.0,
   3.3333333333333335,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   10.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0],
  [0.0,
   0.0,
   3.3333333333333335,
   3.3333333333333335,
   3.3333333333333335,
   0.0,
   0.0,
   0.0,
   3.3333333333333335,
   0.0,
   0.0,
   0.0,
   3.3333333333333335,
   0.0,
   3.3333333333333335,
   3.3333333333333335,
   3.3333333333333335,
   3.3333333333333335,
   3.3333333333333335,
   3.3333333333333335,
   0.0,
   0.0],
  [0.0,
   0.0,
   3.3333333333333335,
   0.0,
   0.0,
   0.0,
   0.0,
   3.3333333333333335,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   3.3333333333333335,
   0.0,
   0.0,
   0.0],
  [0.0,
   0.0,
   3.3333333333333335,
   0.0,
   0.0,
   3.3333333333333335,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   3.3333333333333335,
   3.3333333333333335,
   0.0,
   3.3333333333333335,
   0.0,
   0.0,
   3.333333333333

In [45]:
#model by model

In [46]:
model_mean=np.mean(pinev,axis=0)
model_spread=np.std(pinev,axis=0)
IV=(np.sum((model_spread*model_spread))/22)**.5
IV,model_spread

(1.49110896125784,
 array([1.86338998, 0.        , 1.54504135, 1.54504135, 1.10239638,
        1.10239638, 1.94320397, 1.44337567, 0.80687153, 1.30104125,
        2.03058216, 1.65359457, 1.61374306, 0.80687153, 2.8182811 ,
        1.10239638, 1.30104125, 1.54504135, 1.44337567, 1.30104125,
        1.10239638, 1.30104125]))

In [47]:
#16 30-yr periods MMEs

In [48]:
mme_mean=np.mean(pinev,axis=1)
mme_spread=np.std(pinev,axis=1)
#internal variability mean and spread
mme_mean,np.mean(mme_spread)

(array([0.75757576, 1.66666667, 0.45454545, 0.90909091, 0.60606061,
        0.60606061, 0.75757576, 0.60606061, 1.06060606, 1.36363636,
        0.60606061, 0.60606061, 0.75757576, 0.15151515, 1.36363636,
        0.60606061]),
 1.488695723967351)

In [49]:
#MME mean and spread

In [50]:
mme_model_mean=np.mean(mme_mean)
mme_model_spread=np.std(mme_mean)
mme_model_mean,mme_model_spread

(0.8049242424242424, 0.37390316330302337)

In [51]:
#cross model mean and spread

In [52]:
cross_model_mean=np.mean(model_mean)
cross_model_spread=np.std(model_mean)
cross_model_mean,cross_model_spread

(0.8049242424242425, 0.5036775055671289)

In [53]:
IV,cross_model_spread,mme_model_spread,np.std(pinev),np.std(pinev)-IV

(1.49110896125784,
 0.5036775055671289,
 0.37390316330302337,
 1.5738795900442193,
 0.0827706287863792)

In [54]:
## Calculate thresholds for PI Control runs

In [55]:
def get_thre(loc,y1,y2,prc1,prc2):
    # open data: anomalies (without trend)

    modsst = xr.open_dataset(fdir+'original ts/picontrol_'+loc+'_sst.nc')
    modsst.close()
    modsm2 = xr.open_dataset(fdir+'original ts/picontrol_sm.nc')
    modsm2.close()

    # select period 
    modsst = modsst.sel(year=slice(y1,y2))
    #modsm2 = modsm2.sel(year=slice(y1-1,y2-1))
    modsm2 = modsm2.sel(year=slice(y1,y2))


    mhw_thr = list()
    drg_thr = list()
    models = list()

    # Calculate thresholds for each model
    for ix,i in enumerate(modsm2.model.values):
        tmp1= modsst.sel(model=i).sst.values
        tmp2= modsm2.sel(model=i).sm.values
        
        mhw_thr.append(np.nanpercentile(tmp1,prc1))
        drg_thr.append(np.nanpercentile(tmp2,prc2))

        models.append(i)
    
    return mhw_thr, drg_thr , models

In [56]:
# calculate number of events for control, historical and future
# based on pi control thresholds

def freq_coevents(fdir, runs,loc, mhw_thr, drg_thr, models, y1, y2):
    #pi control
    fi1 = fdir+'original ts/'+runs+'_'+loc+'_sst.nc'
    fi2 = fdir+'original ts/'+runs+'_sm.nc'
    modsst = xr.open_dataset(fi1)
    modsst.close()
    modsm2 = xr.open_dataset(fi2)
    modsm2.close()
    models=modsm2.model

    # select period
    modsst = modsst.sel(year=slice(y1,y2))
    modsm2 = modsm2.sel(year=slice(y1,y2))

    nev = list()
    drt=list()
    mhw=list()

    for ix,i in enumerate(models):
        #print(ix,i)
        tmp1= modsst.sel(model=i).sst.values
        tmp2= modsm2.sel(model=i).sm.values

        a1 = tmp1>=mhw_thr[ix]
        a2 = tmp2<=drg_thr[ix]
        tmp = np.full((len(modsst.year.values),1),1)
        tmp = tmp[a1*a2]
        
        tmp1 = np.full((len(modsst.year.values),1),1)
        tmp1 = tmp1[a1]
        tmp2 = np.full((len(modsst.year.values),1),1)
        tmp2 = tmp2[a2]
        
        nev.append(len(tmp)/30*100)
        mhw.append(len(tmp1)/30*100)
        drt.append(len(tmp2)/30*100)
    freq = np.array(nev) #/(y2-y1+1)
    
    return nev,  mhw, drt #mean no. of events
#freq, np.round(np.nanmean(freq),4), np.round(np.nanstd(freq),4) #mean frequency


In [57]:
def freq_difruns(loc, ny, y0,y,prc1, prc2):
    
    # calculate threshold in pi control
    #y2=2014
    #y1=y2-ny+1
    mhw_thr, drg_thr, models = get_thre(loc,y0,y,prc1,prc2)

    print ('Percentiles: '+str(prc1)+'/'+str(prc2))
    # pi control
    print('piControl: '+str(y0)+'-'+str(y))
    #y2=2014
    #y1=y2-ny+1
    freq0, mfrq, sfrq = freq_coevents(fdir,'picontrol',loc,mhw_thr,drg_thr, models,y0,y)
    print(mfrq,sfrq)

    # historical 2
    y2=2029
    y1=y2-ny+1
    print('\nhistorical 2: '+str(y1)+'-'+str(y2))
    freq1, mfrq1, sfrq1 = freq_coevents(fdir,'histssp585',loc,mhw_thr,drg_thr, models, y1,y2)
    print(mfrq1,sfrq1)

    # historical 2
    y2=2014
    y1=y2-ny+1
    print('\nhistorical 2: '+str(y1)+'-'+str(y2))
    freq, mfrq, sfrq = freq_coevents(fdir,'historical',loc,mhw_thr,drg_thr, models, y1,y2)
    print(mfrq,sfrq)

    # future
    y2=2099
    y1=y2-ny+1
    print('\nfuture: '+str(y1)+'-'+str(y2))
    freq2, mfrq2, sfrq2 = freq_coevents(fdir,'ssp585',loc,mhw_thr,drg_thr, models, y1,y2)
    print(mfrq2,sfrq2)
    
    # future 2
    y2=2044
    y1=y2-ny+1
    print('\nfuture: '+str(y1)+'-'+str(y2))
    freq, mfrq, sfrq = freq_coevents(fdir,'ssp585',loc,mhw_thr,drg_thr, models, y1,y2)
    print(mfrq,sfrq)
    
    return freq0,freq1,freq2,mfrq1, sfrq1,mfrq2, sfrq2

In [58]:
# Frequency of events - detrended data, using pi control threshols
## percentiles 90-10

In [59]:

print ('\nPercentiles: '+str(prc1)+'/'+str(prc2))
print('\nProbable # events in '+str(ny)+' years'+': '+str(ny*0.01))

funev=list()
hinev=list()
test=list()
him=list()
hid=list()
fum=list()
fud=list()
for i in range(int(500/ny)):
    pi,hi,fu,m1,d1,m2,d2 = freq_difruns(loc,ny,i*ny+1, (i+1)*ny, prc1, prc2)
    test.append(pi)
    funev.append(fu)
    hinev.append(hi)
    him.append(m1)
    hid.append(d1)
    fum.append(m2)
    fud.append(d2)
np.mean(funev),np.mean(hinev),np.mean(him),np.mean(hid),np.mean(fum),np.mean(fud)


Percentiles: 90/10

Probable # events in 30 years: 0.3
Percentiles: 90/10
piControl: 1-30
[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0] [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0]

historical 2: 2000-2029
[43.333333333333336, 56.666666666666664, 66.66666666666666, 90.0, 46.666666666666664, 66.66666666666666, 73.33333333333333, 100.0, 100.0, 80.0, 86.66666666666667, 96.66666666666667, 86.66666666666667, 80.0, 70.0, 53.333333333333336, 86.66666666666667, 86.66666666666667, 76.66666666666667, 80.0, 50.0, 80.0] [13.333333333333334, 3.3333333333333335, 93.33333333333333, 33.33333333333333, 20.0, 3.3333333333333335, 26.666666666666668, 96.66666666666667, 100.0, 86.66666666666667, 86.66666666666667, 0.0, 16.666666666666664, 13.333333333333334, 0.0, 0.0, 3.3333333333333335, 43.333333333333336, 20.0, 16.666666666666664, 

[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0] [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0]

historical 2: 2000-2029
[60.0, 46.666666666666664, 66.66666666666666, 83.33333333333334, 76.66666666666667, 70.0, 76.66666666666667, 76.66666666666667, 100.0, 86.66666666666667, 100.0, 76.66666666666667, 90.0, 80.0, 93.33333333333333, 50.0, 86.66666666666667, 80.0, 73.33333333333333, 56.666666666666664, 63.33333333333333, 80.0] [30.0, 13.333333333333334, 86.66666666666667, 40.0, 0.0, 0.0, 26.666666666666668, 73.33333333333333, 100.0, 90.0, 100.0, 0.0, 0.0, 10.0, 0.0, 10.0, 6.666666666666667, 20.0, 50.0, 23.333333333333332, 23.333333333333332, 0.0]

historical 2: 1985-2014
[16.666666666666664, 20.0, 30.0, 46.666666666666664, 40.0, 30.0, 60.0, 30.0, 93.33333333333333, 73.33333333333333, 73.33333333333333, 46.666666666666664, 

[6.666666666666667, 20.0, 33.33333333333333, 76.66666666666667, 30.0, 23.333333333333332, 50.0, 86.66666666666667, 93.33333333333333, 66.66666666666666, 83.33333333333334, 53.333333333333336, 93.33333333333333, 73.33333333333333, 36.666666666666664, 26.666666666666668, 73.33333333333333, 50.0, 16.666666666666664, 23.333333333333332, 13.333333333333334, 30.0] [6.666666666666667, 10.0, 96.66666666666667, 23.333333333333332, 13.333333333333334, 3.3333333333333335, 6.666666666666667, 93.33333333333333, 100.0, 93.33333333333333, 60.0, 3.3333333333333335, 13.333333333333334, 13.333333333333334, 10.0, 0.0, 6.666666666666667, 16.666666666666664, 46.666666666666664, 6.666666666666667, 33.33333333333333, 0.0]

future: 2070-2099
[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0] [6.666666666666667, 3.3333333333333335, 100.0, 66.66666666666666, 13.333333333333334, 0.0, 40.0, 100.0, 100.0, 100.0,

[93.33333333333333, 80.0, 86.66666666666667, 96.66666666666667, 83.33333333333334, 83.33333333333334, 83.33333333333334, 96.66666666666667, 96.66666666666667, 96.66666666666667, 93.33333333333333, 93.33333333333333, 96.66666666666667, 96.66666666666667, 93.33333333333333, 76.66666666666667, 93.33333333333333, 86.66666666666667, 96.66666666666667, 66.66666666666666, 76.66666666666667, 93.33333333333333] [13.333333333333334, 23.333333333333332, 90.0, 23.333333333333332, 10.0, 3.3333333333333335, 36.666666666666664, 86.66666666666667, 96.66666666666667, 93.33333333333333, 90.0, 0.0, 13.333333333333334, 23.333333333333332, 10.0, 3.3333333333333335, 0.0, 6.666666666666667, 6.666666666666667, 10.0, 20.0, 3.3333333333333335]
Percentiles: 90/10
piControl: 301-330
[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0] [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0

[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0] [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0]

historical 2: 2000-2029
[70.0, 93.33333333333333, 70.0, 93.33333333333333, 73.33333333333333, 70.0, 83.33333333333334, 76.66666666666667, 93.33333333333333, 93.33333333333333, 90.0, 93.33333333333333, 86.66666666666667, 80.0, 83.33333333333334, 70.0, 86.66666666666667, 83.33333333333334, 70.0, 76.66666666666667, 63.33333333333333, 80.0] [0.0, 6.666666666666667, 93.33333333333333, 40.0, 0.0, 0.0, 13.333333333333334, 96.66666666666667, 96.66666666666667, 90.0, 90.0, 0.0, 3.3333333333333335, 10.0, 0.0, 0.0, 6.666666666666667, 0.0, 20.0, 10.0, 46.666666666666664, 13.333333333333334]

historical 2: 1985-2014
[20.0, 56.666666666666664, 40.0, 63.33333333333333, 33.33333333333333, 30.0, 60.0, 33.33333333333333, 83.33333333333334, 8

(43.62689393939394,
 23.390151515151516,
 76.14583333333334,
 30.397727272727273,
 100.0,
 43.62689393939394)

In [60]:
loc,ny,np.mean(model_mean),np.mean(hinev),np.mean(funev)

('cclme', 30, 0.8049242424242425, 23.390151515151516, 43.62689393939394)

In [61]:
N=int(500/ny)
N

16

In [62]:
#hist - picontrol FAR
diff=list()
for k in range(len(pinev)):
    zip_obj=zip(hinev[k],pinev[k])
    for i , j in zip_obj:
            diff.append(i-j)
    #diff mean, model spread
diff_a=np.asarray(diff).reshape(N,22)
diff_mean=np.mean(diff_a,axis=1)
diff_spread=np.std(diff_mean)
diff_mean.mean(),diff_spread

(22.585227272727273, 1.7062018467155442)

In [63]:
hinev_a=np.asarray(hinev).reshape(N,22).sum(axis=1)
pinev_a=np.asarray(pinev).reshape(N,22).sum(axis=1)
far=(hinev_a-pinev_a)/hinev_a
FAR=np.mean(far)
sd=np.std(far)
FAR,sd



(0.9652945479782704, 0.017346302500950553)

In [64]:
import statsmodels.api as sm
import statsmodels as sm
import pandas
from patsy import dmatrices
cu,cl=sm.stats.proportion.proportion_confint( FAR,1, alpha=0.05, method='normal') #'binom_test''normal'
FAR,cu,cl

(0.9652945479782704, 0.6065569711404133, 1.0)

In [65]:
#future - hist change
#cross-mme spread
diff=list()
for k in range(len(pinev)):
    zip_obj=zip(funev[k],hinev[k])
    for i , j in zip_obj:
            diff.append(i-j)
    #diff mean, model spread
diff_a=np.asarray(diff).reshape(N,22)
diff_mean=np.mean(diff_a,axis=1)
diff_spread=np.std(diff_mean)
diff_mean.mean(),diff_spread

(20.236742424242422, 1.905998540144283)

In [66]:
#cross-model spread
diff=list()
for k in range(len(pinev)):
    zip_obj=zip(funev[k],hinev[k])
    for i , j in zip_obj:
            diff.append(i-j)
    #diff mean, model spread
diff_mean=np.mean(diff)
diff_spread=np.std(diff)-IV
diff_mean,diff_spread

(20.236742424242422, 21.637882935404605)

In [67]:
funev_a=np.asarray(funev).reshape(N,22).sum(axis=1)
pinev_a=np.asarray(pinev).reshape(N,22).sum(axis=1)
far=(funev_a-pinev_a)/funev_a
FAR=np.mean(far)
sd=np.std(far)
FAR=np.mean(far)
cu,cl=sm.stats.proportion.proportion_confint(FAR, 1, alpha=0.05, method='normal') #'binom_test''normal'
FAR,cu,cl,sd

(0.9814823207147249, 0.7172520251011603, 1.0, 0.008886887613228573)

In [68]:
#individual extremes
np.mean(him),np.mean(fum),np.mean(hid),np.mean(fud)

(76.14583333333334, 100.0, 30.397727272727273, 43.62689393939394)

In [69]:
him_a=np.asarray(him).reshape(N,22).sum(axis=1)
pim_a=np.asarray(pim).reshape(N,22).sum(axis=1)
far=(him_a-pim_a)/him_a
FAR=np.mean(far)
sd=np.std(far)
FAR=np.mean(far)
cu,cl=sm.stats.proportion.proportion_confint(FAR, 1, alpha=0.05, method='normal') #'binom_test''normal'
FAR,cu,cl,sd


(0.8685886116674988, 0.20641532234889126, 1.0, 0.003290271860507996)

In [70]:
diff=list()
for k in range(len(pinev)):
    zip_obj=zip(him[k],pim[k])
    for i , j in zip_obj:
            diff.append(i-j)
    #diff mean, model spread
diff_a=np.asarray(diff).reshape(N,22)
diff_mean=np.mean(diff_a,axis=1)
diff_spread=np.std(diff_mean)
diff_mean.mean(),diff_spread

(66.14583333333334, 1.9550509902368447)

In [71]:
diff=list()
for k in range(len(pinev)):
    zip_obj=zip(fum[k],him[k])
    for i , j in zip_obj:
            diff.append(i-j)
    #diff mean, model spread
diff_a=np.asarray(diff).reshape(N,22)
diff_mean=np.mean(diff_a,axis=1)
diff_spread=np.std(diff_mean)
diff_mean.mean(),diff_spread

(23.854166666666664, 1.9550509902368471)

In [72]:
diff=list()
for k in range(len(pinev)):
    zip_obj=zip(fum[k],him[k])
    for i , j in zip_obj:
            diff.append(i-j)
    #diff mean, model spread
diff_mean=np.mean(diff)
diff_spread=np.std(diff)-IV
diff_mean,diff_spread

(23.854166666666664, 15.098110956730277)

In [73]:
him_a=np.asarray(fum).reshape(N,22).sum(axis=1)
pim_a=np.asarray(pim).reshape(N,22).sum(axis=1)
far=(him_a-pim_a)/him_a
FAR=np.mean(far)
sd=np.std(far)
FAR=np.mean(far)
cu,cl=sm.stats.proportion.proportion_confint(FAR, 1, alpha=0.05, method='normal') #'binom_test''normal'
FAR,cu,cl,sd

(0.9, 0.3120108046379837, 1.0, 0.0)

In [74]:
him_a=np.asarray(hid).reshape(N,22).sum(axis=1)
pim_a=np.asarray(pid).reshape(N,22).sum(axis=1)
far=(him_a-pim_a)/him_a
FAR=np.mean(far)
sd=np.std(far)
FAR=np.mean(far)
cu,cl=sm.stats.proportion.proportion_confint(FAR, 1, alpha=0.05, method='normal') #'binom_test''normal'
FAR,cu,cl,sd

(0.6693063163582893, 0.0, 1.0, 0.023832720505877775)

In [75]:
diff=list()
for k in range(len(pinev)):
    zip_obj=zip(hid[k],pid[k])
    for i , j in zip_obj:
            diff.append(i-j)
    #diff mean, model spread
diff_a=np.asarray(diff).reshape(N,22)
diff_mean=np.mean(diff_a,axis=1)
diff_spread=np.std(diff_mean)
diff_mean.mean(),diff_spread

(20.39772727272727, 2.205362105326617)

In [76]:
diff=list()
for k in range(len(pinev)):
    zip_obj=zip(fud[k],hid[k])
    for i , j in zip_obj:
            diff.append(i-j)
    #diff mean, model spread
diff_a=np.asarray(diff).reshape(N,22)
diff_mean=np.mean(diff_a,axis=1)
diff_spread=np.std(diff_mean)
diff_mean.mean(),diff_spread

(13.229166666666668, 1.8430424336068063)

In [77]:
diff=list()
for k in range(len(pinev)):
    zip_obj=zip(fud[k],hid[k])
    for i , j in zip_obj:
            diff.append(i-j)
    #diff mean, model spread
diff_mean=np.mean(diff)
diff_spread=np.std(diff)-IV
diff_mean,diff_spread

(13.229166666666664, 21.304823863392254)

In [78]:
him_a=np.asarray(fud).reshape(N,22).sum(axis=1)
pim_a=np.asarray(pid).reshape(N,22).sum(axis=1)
far=(him_a-pim_a)/him_a
FAR=np.mean(far)
sd=np.std(far)
FAR=np.mean(far)
cu,cl=sm.stats.proportion.proportion_confint(FAR, 1, alpha=0.05, method='normal') #'binom_test''normal'
FAR,cu,cl,sd

(0.7699979257960695, 0.0, 1.0, 0.013453015650550303)